# Ingest Y23 10X h5ad counts matrix, merge with annotations, and write to csv


In [ ]:
# Define constants
name_root = 'Y23_Ingest'
results_path = 'Products/'+name_root+'/'


In [ ]:
# Import libraries and define functions

# Initialize
import math
import time
import numpy as np
import pandas as pd
import anndata as ad
import os 
import glob
import json
import textwrap

# File serializer
def serial(pad): # generate a zero padded serial number string
    sn_file = 'SN.txt'
    f = open(sn_file, "r")
    n = int(f.read())    
    f = open(sn_file, "w")
    f.write(str(n+1))
    f.close()
    return str(n).zfill(pad)

# Call to halt script
class StopExecution(Exception):
    def _render_traceback_(self):
        return []

# Shorten class, subclass, supertype or cluster label
def shortener(s):
    head = s.split(' ')[0]
    tail = s.split(' ')[-1]
    tail = tail.replace('Gaba','GABA')
    if tail != 'NN': return head+'_'+tail
    return head+'_'+s.split(' ')[1]

# Form table of all gene IDs and symbols
all_genes_file = 'Resources/Y23_final_gene_list.csv'
all_genes_table = pd.read_csv(all_genes_file).drop(['name','comment'],axis=1)

# Form lists of gene IDs and symbols
#chosen_gene_set = 'mNPs' # 'mNPs','mNMods' or 'mPCs'
#gene_symbols = list(pd.read_csv('Genelists/Roots/'+chosen_gene_set+'.csv', header = None)[0])
#gene_symbols = [x.capitalize() for x in gene_symbols]
geneset = 'nMod' # 'NP' or 'nMod'
gene_symbols = list(pd.read_csv('Resources/All_mouse_'+geneset+'_genes.csv')['Symbol'])
gene_symbols = [x for x in gene_symbols if x in list(all_genes_table['gene_symbol'])]
gene_ids = list(all_genes_table.set_index('gene_symbol').loc[gene_symbols]['gene_identifier'])
print('Ingesting set '+geneset+',',len(gene_symbols),'genes')

pc_gene_symbols = list(pd.read_csv('Resources/All_mouse_PC_genes.csv', header = None)[0])
pc_gene_symbols = [x.capitalize() for x in pc_gene_symbols]
pc_gene_symbols = [x for x in pc_gene_symbols if x in list(all_genes_table['gene_symbol'])]
pc_gene_ids = list(all_genes_table.set_index('gene_symbol').loc[pc_gene_symbols]['gene_identifier'])

# Read annotation table and form useful derivatives
annots_table = pd.read_csv('Resources/Y23_annots_table.csv')
read_column_list = [x for x in list(annots_table['read_columns']) if type(x) == str]
read_type_list = [x for x in list(annots_table['read_type']) if type(x) == str]
destination_column_list = [x for x in list(annots_table['destination_columns']) if type(x) == str]
read_dict = dict(zip(read_column_list, read_type_list))

# Read in taxon table and build class-type dictionary
taxon_table = pd.read_csv('Resources/Y23_taxon_table.csv')
type_dict = taxon_table.set_index('class').to_dict('dict')['type']

# Read, filter and edit metadata file
meta = pd.read_csv('Resources/Y23_final_cell_metadata_with_cluster_annotation.csv',
                   usecols=read_column_list,dtype=read_dict,index_col='cell_label')
print(len(meta),'cells')

# Add type column
meta.insert(0,'type',[type_dict[s] for s in list(meta['class'])])

# Rename some metadata columns
df = meta.copy()
lst = list(df.columns)
lst = [x.replace('region_of_interest','region') for x in lst]
df.columns = lst
df.rename(columns={'class': 'Y23_class',
                   'class_color': 'Y23_class_color',
                   'neurotransmitter': 'class',
                   'neurotransmitter_color': 'class_color'}, inplace=True)
meta = df.copy()

# Edit some metadata column data
df = pd.read_csv('Resources/Y23_transmitters.csv')
transmitter_dict = dict(zip(list(df['short']), list(df['class'])))
df = meta.copy()
df['class'] = [x if str(x) != 'nan' else 'NN' for x in list(df['class'])]
df['class'] = [transmitter_dict[s] for s in list(df['class'])]
df['region_order'] = [f"{x:03d}" for x in list(df['region_order'])]
df.insert(0,'region',df['region_order']+' '+df['region_acronym'])
df.drop(['region_acronym','region_order'], axis=1, inplace=True)
meta = df.copy()

# Add type column
#meta.insert(0,'type',[type_dict[s] for s in list(meta['class'])])

# Add row ordinal list as last metadata column
meta['ordinal'] = list(np.arange(len(meta)))

# Write out metadata summary table
print(len(meta.columns),'metadata columns after editing')
df = meta.drop_duplicates(meta.columns[2:-2])
df.to_csv('Resources/Y23_metadata_summary.csv', index=False)
print(len(df),'categories from metadata columns: region, class, subclass, supertype and cluster')


In [ ]:
# Filter and merge metadata and data for all regions/platforms 

# Use glob to find all the h5ad files in the folder 
path = '/Users/stephens/Primary_scRNAseq_Data/Y23_final'
h5ad_files = glob.glob(os.path.join(path, "*.h5ad"))
files = len(h5ad_files)
loop_start = time.time()

# Read, sift, merge and concatenate all h5ad files in path
for i, h5ad_file in enumerate(h5ad_files): 
    # Open one h5ad file
    source = ad.read_h5ad(h5ad_file)
    print(h5ad_file, '('+str(i+1)+'/'+str(files)+')',
          '\nSource matrix shape:', source.shape[0],'X', source.shape[1])
    # Read and sift
    start = time.time()
    df = source.to_df().astype(np.uint16)[pc_gene_ids]

    sums = df.sum(axis=1)
    genes = df[df > 0].count(axis=1)
    sifted = df[gene_ids]

    print('Sifted matrix shape:', sifted.shape[0],'X', sifted.shape[1],
          'Read time:',"%.1f seconds" % (time.time()-start),
          'Memory usage:',"%.1f MB," % (sum(sifted.memory_usage(index=True,deep=True))/1e6),
          'Max Value:', max(sifted.max()))
    # Merge with annotation metadata
    df = all_genes_table.set_index('gene_identifier').loc[gene_ids].join(sifted.T, how='inner')
    df = df.set_index('gene_symbol').T
    df.insert(loc=0, column='counts_sum', value=sums)
    df.insert(loc=0, column='genes_mapped', value=genes)
    df = meta.join(df, how='inner')#.set_index(list(meta.columns))
    
    # Concatenate
    if i==0: merged = df
    else: merged = pd.concat([merged, df])
    print('Merged matrix shape:', merged.shape[0],'X', merged.shape[1],
          'Memory usage:',"%.1f MB" % (sum(merged.memory_usage(index=True,deep=True))/1e6),'\n')
    # Close the h5ad file
    source.file.close()

print('Total read and merge time:',"%.1f seconds\n" % (time.time()-loop_start))
    

In [ ]:
# Tidy up

start = time.time()
# Rearrange annotation columns
annots = ['type','region','class','subclass','supertype','cluster','library_method','Y23_class','class_color',
          'Y23_class_color','subclass_color','supertype_color','cluster_color','region_color','ordinal',
          'genes_mapped','counts_sum']
df = merged.copy()
df.reset_index(drop=True,inplace=True)
df.set_index(annots,inplace=True)

# Sort merged result for later convenience
print('Sorting merged result table')
df.sort_index(level=['region','class','subclass','supertype','cluster'], inplace=True)
print('Elapsed write time:',"%.1f seconds" % (time.time()-start))


In [ ]:
# Sanity check
df[['Cck','Npy','Sst','Grp','Cckbr']].sum()

In [ ]:
# Compile support dictionary 
support_dict = {}
support_dict.update({'input_files': h5ad_files})
support_dict.update({'hierarchy': '34 classes, 338 subclasses, 1,201 supertypes and 5,322 clusters'})
support_dict.update({'annotation_labels': annots})

# Set up output paths
sn = serial(5)
json_path = 'Products/Y23_'+geneset+'_support_'+sn+'.json'
print('              Support path:', json_path)
csv_path = 'Products/Y23_'+geneset+'_counts_'+sn+'.csv.zip'
print('Expression data write path:', csv_path)

# Write out support dictionary
with open(json_path, 'w') as fp:
    json.dump(support_dict, fp)


In [ ]:
# Write out merged result
start = time.time()
print('Writing: %.1f MB to' % (sum(df.memory_usage(index=True,deep=True))/1e6), csv_path)
df.to_csv(csv_path, chunksize=100000)
print('Elapsed write time:',"%.1f seconds" % (time.time()-start))


In [ ]:
# Readback
print(csv_path)
df = pd.read_csv(csv_path)


In [ ]:
# Sanity check
df[['Cck','Npy','Vip','Cckbr','Kng1','Bdkrb1','Bdkrb2']].sum()


In [ ]:
# Generate and write a new 'raw' taxon table, if desired
taxa = ['type','class','subclass','supertype','cluster']
#df = meta.iloc[:,0:0].reset_index().copy()
df = meta[taxa].drop_duplicates()
for taxon in taxa:
    df[taxon+'_short'] = list(map(shortener,df[taxon]))
    print(taxon)
df.to_csv('Resources/Y23_taxon_table_raw.csv')
